![alt text for screen readers](https://intro-to-btt-using-python-assets.s3.amazonaws.com/bladesight_logo_horizontal_ORIGINAL.jpg).
# Chapter 5: Multiple probes

## Dependencies

In [ ]:
# Run this cell if you have not installed the `bladesight` package yet
%pip install bladesight
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If plotly is not installed
%pip install plotly
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

In [ ]:
# If Numba is not installed
%pip install numba
## NBNBNB! You may need to restart the kernel after installing the package! If you 
# installed it through the Kernel, you can skip this cell.

## Imports

In [ ]:
from bladesight import Datasets
from bladesight.btt.triggering_criteria import threshold_crossing_hysteresis_pos
from bladesight.btt.aoa import transform_ToAs_to_AoAs, transform_prox_AoAs_to_blade_AoAs
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from typing import List

In [ ]:
ds = Datasets["data/intro_to_btt/intro_to_btt_ch05"]
df_opr_zero_crossings = ds['table/opr_zero_crossings']
df_prox_1 = ds['table/prox_1_toas']
df_prox_2 = ds['table/prox_2_toas']
df_prox_3 = ds['table/prox_3_toas']
df_prox_4 = ds['table/prox_4_toas']


## Assembling probe level merged DataFrames

In [ ]:

probes_dfs = []
blade_probes_dfs = []
B = 5

for df_prox_toas in [df_prox_1, df_prox_2, df_prox_3, df_prox_4]:
    df_prox = transform_ToAs_to_AoAs(
        df_opr_zero_crossings, 
        df_prox_toas, 
    )
    
    blade_probes_dfs.append(
        transform_prox_AoAs_to_blade_AoAs(
            df_prox, 
            B
        )
    )

In [ ]:
print("df_prox all blades")
df_prox.head(15)

In [ ]:
for blade_no in range(5):
    print(f"Blade {blade_no}:")
    print(blade_probes_dfs[-1][blade_no].head(3))

In [ ]:
def align_blade_AoAs_along_revolutions(
    prox_AoA_dfs : List[pd.DataFrame]
) -> pd.DataFrame:
    """This function aligns the AoA DataFrames (from the 
        transform_prox_AoAs_to_blade_AoAs function) along
        the shaft revolutions.

        The function returns a DataFrame with the ToA
        and AoA values for each blade having a column.

    Args:
        prox_AoA_dfs (List[pd.DataFrame]): A list of DataFrames
        where each DataFrame contains the ToA and AoA values
        for a single blade from a proximity probe.

    Returns:
        pd.DataFrame: A DataFrame where every row contains the
        data pertaining to a single shaft revolution and every 
        blade's ToA and AoA values are in its own column respectively.
    """
    df_blades_aligned = prox_AoA_dfs[0]
    # Rename the ToA and AoA columns to include the blade number
    df_blades_aligned = df_blades_aligned.rename(columns={"ToA":"ToA_1", "AoA":"AoA_1"})
    for i, df_blade in enumerate(prox_AoA_dfs[1:]):
        df_blades_aligned = df_blades_aligned.merge(
            df_blade[["n", "ToA", "AoA"]].rename(columns={"ToA":"ToA_"+str(i+2), "AoA":"AoA_"+str(i+2)}),
            how="outer",
            on="n"
        )
    return df_blades_aligned


In [ ]:
blade_dfs_recombined = []
for prox_list_AoAs in blade_probes_dfs:
    blade_dfs_recombined.append(
        align_blade_AoAs_along_revolutions(prox_list_AoAs)
    )

In [ ]:
blade_dfs_recombined[-1].head(3)

# Stack plot

In [ ]:
def create_stack_plot_df(df_blades_aligned : pd.DataFrame) -> pd.DataFrame:
    """ This function creates a DataFrame that shows the consecutive
    difference between adjacent blades for each shaft revolution.
    
    Args:
        df_blades_aligned (pd.DataFrame): A DataFrame where every row contains the
        data pertaining to a single shaft revolution and every
        blade's ToA and AoA values are in its own column respectively. This
        is the output of the `align_blade_AoAs_along_revolutions` function.

    Returns:
        pd.DataFrame: A DataFrame where every row contains the
        data pertaining to a single shaft revolution and every
    """
    all_aoa_columns = sorted([
        i for i in df_blades_aligned.columns 
        if i.startswith("AoA_")
    ])
    B = len(all_aoa_columns)
    stack_plot_diffs = {}
    stack_plot_diffs["n"] = df_blades_aligned["n"].to_numpy()
    for blade_no in range(B - 1):
        further_blade_name = all_aoa_columns[blade_no + 1]
        closer_blade_name = all_aoa_columns[blade_no]
        arr_blade_diffs = (
            df_blades_aligned[further_blade_name] 
            - df_blades_aligned[closer_blade_name]
        ).to_numpy()
        
        stack_plot_diffs[closer_blade_name] = arr_blade_diffs
    further_blade_name = all_aoa_columns[0]
    closer_blade_name = all_aoa_columns[B - 1]
    arr_blade_diffs = (
        df_blades_aligned[further_blade_name].to_numpy()[1:] + 2*np.pi 
        - df_blades_aligned[closer_blade_name].to_numpy()[:-1]
    )
    # Append a NaN to the end of arr_blade_diffs
    # so that it has the same length as the other
    # blade difference arrays
    arr_blade_diffs = np.append(arr_blade_diffs, [None])
    stack_plot_diffs[closer_blade_name] = arr_blade_diffs
    return pd.DataFrame(stack_plot_diffs)


In [ ]:
stack_plot_dfs = []
for df in blade_dfs_recombined:
    stack_plot_dfs.append(create_stack_plot_df(df))
print(stack_plot_dfs[0].median())

In [ ]:
fig = go.Figure()
x = np.arange(len(stack_plot_dfs[0]))
for prox_no, df_prox_stack in enumerate(stack_plot_dfs):
    stack_plot_median = df_prox_stack.iloc[:, 1:].median()
    fig.add_trace(
        go.Scatter(
            x=x,
            y=stack_plot_median,
            mode="lines",
            name=f"Proximity probe {prox_no}"
        )
    )
fig.update_layout(
    title="Stack plot for proximity probe",
    xaxis_title="Blade No",
    yaxis_title="AoA difference [rad]",
    xaxis=dict(
        tickmode="array",
        tickvals=x,
        ticktext=[str(i) for i in stack_plot_median.index]
    )
)
fig.show()

### Offsetting the stack plot values

In [ ]:
def shift_AoA_column_headings(
    aoa_column_headings : List[str], 
    shift_by : int
) -> List[str]:
    """This function shifts the columns headings of the AoA
    such that the first column heading represents the first blade
    arriving at the first probe.

    Args:
        arr_values (np.ndarray): The array of values to be shifted
        shift_by (int): The number of positions to shift the values
        in the array by.

    Returns:
        np.ndarray: The shifted array of values.
    """
    if shift_by >= len(aoa_column_headings):
        raise ValueError("shift_by must be less than the number blades in aoa_column_headings")
    return (
        list(aoa_column_headings)[shift_by:] 
        + list(aoa_column_headings)[:shift_by]
    )

In [ ]:
probe_1_col_headings = ["AoA_1", "AoA_2", "AoA_3", "AoA_4", "AoA_5"]
print("Shift by 0:", shift_AoA_column_headings(probe_1_col_headings, 0))
print("Shift by 1:", shift_AoA_column_headings(probe_1_col_headings, 1))
print("Shift by 2:", shift_AoA_column_headings(probe_1_col_headings, 2))
print("Shift by 3:", shift_AoA_column_headings(probe_1_col_headings, 3))
print("Shift by 4:", shift_AoA_column_headings(probe_1_col_headings, 4))


In [ ]:
def rename_df_columns_for_alignment(
    df_to_align : pd.DataFrame,
    global_column_headings : List[str],
    shift_by : int
) -> pd.DataFrame:
    """This function performs two tasks. Firstly, it determines the mapping 
    between the global column headings and the column headings `df_to_align`. 
    Secondly, it renames and re-orders the columns in df_to_align such that 
    the columns appear in the same order in df_to_align.

    Args:
        df_to_align (pd.DataFrame): The DataFrame whose columns are to be
            renamed and re-ordered.
        global_column_headings (List[str]): The column headings
            to which the columns in df_to_align should be mapped. This
            will normally be AoA or ToA column headings.
        shift_by (int): The number of positions to shift the values
            in the array by.

    Returns:
        pd.DataFrame: The DataFrame with the renamed and re-ordered columns.
    """
    # Create a dictionary that maps the column headings in df_to_align
    # to the global column headings
    shifted_dataframe_columns = shift_AoA_column_headings(
        global_column_headings, 
        shift_by
    )
    column_headings_to_rename = {
        local_col : global_col
        for local_col, global_col 
        in zip(
            shifted_dataframe_columns,
            global_column_headings
        )
    }
    original_column_order = list(df_to_align.columns)
    # Rename the columns in df_to_align
    df_to_align = df_to_align.rename(
        columns=column_headings_to_rename
    )
    return df_to_align[original_column_order]

In [ ]:
stack_plot_dfs_aligned = []
offsets = [0,1,1,1]

for df, offset in zip(stack_plot_dfs, offsets):
    df_aoas_shifted = rename_df_columns_for_alignment(
        df, 
        ["AoA_1", "AoA_2", "AoA_3", "AoA_4", "AoA_5"], 
        offset
    )
    stack_plot_dfs_aligned.append(
        df_aoas_shifted
    )

In [ ]:
fig = go.Figure()
x = np.arange(len(stack_plot_dfs_aligned[0]))
x_names = None
for prox_no, df_prox_stack in enumerate(stack_plot_dfs_aligned):
    stack_plot_median = df_prox_stack.iloc[:, 1:].median()
    if x_names is None:
        x_names = stack_plot_median.index.to_list()
    fig.add_trace(
        go.Scatter(
            x=x,
            y=stack_plot_median,
            mode="lines",
            name=f"Proximity probe {prox_no}"
        )
    )
fig.update_layout(
    title="Stack plot for proximity probe",
    xaxis_title="Blade No",
    yaxis_title="AoA difference [rad]",
    xaxis=dict(
        tickmode="array",
        tickvals=x,
        ticktext=[str(i) for i in x_names]
    )
)
fig.show()

### Calculating the shift based on probe spacing 

In [ ]:
def predict_probe_offset(
        df_probe_AoAs : pd.DataFrame,
        starting_aoa : float, 
        prox_probe_relative_distance : float,
    ) -> int:
    """This function calculates the offset that needs to be applied to
    the AoA columns of the current probe to align them with the first
    probe.

    Args:
        df_probe_AoAs (pd.DataFrame): A DataFrame where
            every row contains the data pertaining to a single shaft
            revolution and every blade's ToA and AoA values are in its
            own column respectively. This is the output of the
            `align_blade_AoAs_along_revolutions` function.
        starting_aoa (float): The mean AoA of the blade you want to
            project forward and identify in df_probe_AoAs. In radians.
        prox_probe_relative_distance (float): The relative distance
            between the current probe and the first probe. In radians.


    Returns:
        int: The blade offset that needs to be applied to the AoA values
            in df_probe_AoAs to align it to the blade in starting_aoa
    """
    predicted_blade_position = (
        starting_aoa 
        + prox_probe_relative_distance
    ) % (2*np.pi)
    all_aoa_columns = sorted([
        i for i in df_probe_AoAs.columns 
        if i.startswith("AoA_")
    ])
    current_probe_median_AoAs = df_probe_AoAs[all_aoa_columns].median()
    err_aoa = np.abs(current_probe_median_AoAs - predicted_blade_position)
    offset = np.argmin(err_aoa)
    return offset


In [ ]:
probe_1_blade_1_AoA = blade_dfs_recombined[0]["AoA_1"].median()
probe_spacings = np.deg2rad(np.array([0, 19.34, 19.34*2, 19.34*3]))
for i, (df_probe_AoAs, probe_spacing) in enumerate(zip(blade_dfs_recombined, probe_spacings)):
    probe_offset = predict_probe_offset(
        df_probe_AoAs,
        probe_1_blade_1_AoA,
        probe_spacing
    )
    print(f"Probe {i + 1 }:", probe_offset)

## Assembling global rotor level merged DataFrames


In [ ]:
def assemble_rotor_AoA_dfs(
    prox_aligned_dfs : List[pd.DataFrame], 
    probe_spacing : List[float]
) -> List[pd.DataFrame]:
    """This function assembles the rotor blade AoA DataFrames. In other
    words, this function receives the grouped AoA DataFrames from each
    probe, the one calculated by `align_blade_AoAs_along_revolutions` and 
    shifts the AoA values of each probe such that the first
    blade arriving at the first probe is aligned with the first blade
    arriving at the first probe. 

    We then assemble B DataFrames containing only all the information
    from a single blade over every probe.

    Args:
        prox_aligned_dfs (List[pd.DataFrame]): A list of DataFrames
            where each DataFrame contains the ToAs and AoAs of a single
            blade from a proximity probe. Each DataFrame is the output
            of the `align_blade_AoAs_along_revolutions` function. 
        probe_spacing (List[int]): A list of relative probe spacing
            between the first probe and every other probe. There are one
            less value in this list than in prox_aligned_dfs.

    Returns:
        List[pd.DataFrame]: A list of DataFrames where each DataFrame
            contains the ToAs and AoAs of a single blade over all
            the proximity probes.
    """
    all_aoa_columns = sorted([
        i for i in prox_aligned_dfs[0].columns 
        if i.startswith("AoA_")
    ])
    all_toa_columns = sorted([
        i for i in prox_aligned_dfs[0].columns 
        if i.startswith("ToA_")
    ])
    remaining_columns = [
        i for i in prox_aligned_dfs[0].columns 
        if not i.startswith("ToA_") and not i.startswith("AoA_")
    ]
    B = len(all_aoa_columns)
    P = len(prox_aligned_dfs)
    if P  - 1 != len(probe_spacing):
        raise ValueError(
            "The number of proximity probes must be "
            "one less than the number of probe spacings"
        )
    rotor_blade_dfs = []
    for b in range(1, B+1):
        columns_to_copy = remaining_columns + [f"ToA_{b}", f"AoA_{b}"]
        rename_dict = {
            f"ToA_{b}" : "ToA_p1",
            f"AoA_{b}" : "AoA_p1"
        }
        rotor_blade_dfs.append(
            prox_aligned_dfs[0][columns_to_copy]
            .copy(deep=True)
            .rename(
                columns=rename_dict
            )
        )
    blade_1_probe_1_median = rotor_blade_dfs[0]["AoA_p1"].median()
    for iter_count, (df_probe_AoA, probe_offset) in enumerate(
            zip(prox_aligned_dfs[1:], probe_spacing)
        ):
        probe_no = iter_count + 2
        probe_offset = predict_probe_offset(
            df_probe_AoA,
            blade_1_probe_1_median,
            probe_offset
        )
        df_probe_AoAs_aligned = rename_df_columns_for_alignment(
            df_probe_AoA,
            all_aoa_columns,
            probe_offset
        )
        df_probe_AoAs_aligned = rename_df_columns_for_alignment(
            df_probe_AoAs_aligned,
            all_toa_columns,
            probe_offset
        )
        for b in range(1, B+1):
            columns_to_merge = ["n", f"ToA_{b}", f"AoA_{b}"]
            rename_dict = {
                f"ToA_{b}" : f"ToA_p{probe_no}",
                f"AoA_{b}" : f"AoA_p{probe_no}"
            }
            rotor_blade_dfs[b - 1] = rotor_blade_dfs[b - 1].merge(
                df_probe_AoAs_aligned[columns_to_merge].rename(
                    columns=rename_dict
                ),
                how="outer",
                on="n"
            )
    return rotor_blade_dfs

In [ ]:
prox_relative_distances = np.cumsum(np.deg2rad(np.array([9.67*2, 9.67*2, 9.67*2])))
rotor_blade_AoA_dfs = assemble_rotor_AoA_dfs(
    prox_aligned_dfs=blade_dfs_recombined,
    probe_spacing=prox_relative_distances
)

In [ ]:
rotor_blade_AoA_dfs[0].head(3)

In [ ]:
rotor_blade_AoA_dfs[-1].head(3)

Can you spot the resonances below? 👇

In [ ]:
fig = go.Figure()

for probe in range(1, 5):
    fig.add_trace(
        go.Scatter(
            x=rotor_blade_AoA_dfs[0]["n"],
            y=(
                rotor_blade_AoA_dfs[0][f"AoA_p{probe}"] 
                - rotor_blade_AoA_dfs[0][f"AoA_p{probe}"].mean()
            ),
            mode="lines",
            name=f"Probe {probe}"
        )
    )

fig.update_layout(
    title="AoAs for blade 1",
    xaxis_title="Revolution number",
    yaxis_title="AoA [rad] (normalised)",
)
fig.show()

In [ ]:
(rotor_blade_AoA_dfs[0][f"AoA_p{probe}"] 
- rotor_blade_AoA_dfs[0][f"AoA_p{probe}"].mean())

## Coding exercises

### 1. One function to rule them all

In [ ]:
def get_rotor_blade_AoAs(
    df_opr_zero_crossings : pd.DataFrame,
    prox_probe_toas : List[pd.DataFrame],
    probe_spacings : List[float],
    B : int
) -> List[pd.DataFrame]:
    ...
    # Please complete me

### 2. Predicting the probe spacing

In [ ]:
def predict_probe_spacing(
    df_prox_1_AoAs : pd.DataFrame,
    df_prox_2_AoAs : pd.DataFrame,
) -> float:
    ...
    # Please complete me